In [ ]:
!pip -q install langchain-groq duckduckgo-search
!pip -q install -U langchain_community tiktoken langchainhub
!pip -q install -U langchain langgraph tavily-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.1/417.1 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.0/91.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.5/136.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 4.4 MB/s eta 0:00:00


In [ ]:
!pip show langgraph

Name: langgraph
Version: 0.3.18
Summary: Building stateful, multi-actor applications with LLMs
Home-page: https://www.github.com/langchain-ai/langgraph
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: langchain-core, langgraph-checkpoint, langgraph-prebuilt, langgraph-sdk
Required-by: 


In [ ]:
import os

os.environ["GROQ_API_KEY"] = "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"
os.environ["TAVILY_API_KEY"] = "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"


In [ ]:
from langchain_groq import ChatGroq

GROQ_LLM = ChatGroq(
            model="llama3-70b-8192",
        )

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate

from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser

In [ ]:

def write_markdown_file(content, filename):
  """Writes the given content as a markdown file to the local directory.

  Args:
    content: The string content to write to the file.
    filename: The filename to save the file as.
  """
  with open(f"{filename}.md", "w") as f:
    f.write(content)


In [ ]:
#Categorize EMAIL
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a Email Categorizer Agent You are a master at understanding what a customer wants when they write an email and are able to categorize it in a useful way

     <|eot_id|><|start_header_id|>user<|end_header_id|>
    Conduct a comprehensive analysis of the email provided and categorize into one of the following categories:
        price_equiry - used when someone is asking for information about pricing \
        customer_complaint - used when someone is complaining about something \
        product_enquiry - used when someone is asking for information about a product feature, benefit or service but not about pricing \\
        customer_feedback - used when someone is giving feedback about a product \
        off_topic when it doesnt relate to any other category \


            Output a single cetgory only from the types ('price_equiry', 'customer_complaint', 'product_enquiry', 'customer_feedback', 'off_topic') \
            eg:
            'price_enquiry' \

    EMAIL CONTENT:\n\n {initial_email} \n\n
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["initial_email"],
)

email_category_generator = prompt | GROQ_LLM | StrOutputParser()

EMAIL = """HI there, \n
I am emailing to say that I had a wonderful stay at your resort last week. \n

I really appreaciate what your staff did

Thanks,
Paul
"""

result = email_category_generator.invoke({"initial_email": EMAIL})

print(result)

'customer_feedback'


In [ ]:
# Summary (Summarize if >25 messages)
summary_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are an Email Summarizer Agent. Your task is to condense long emails while preserving key information.
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Summarize this email into a concise version if it exceeds 25 messages.
    Maintain all crucial details including requests, complaints, or important context.
    Return only the summarized version. If under 25 messages, return the original text.

    EMAIL CONTENT:\n\n {initial_email} \n\n
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email"],
)

email_summarizer = summary_prompt | GROQ_LLM | StrOutputParser()

# Example Usage 1
EMAIL = """HI there, \n
I am emailing to say that I had a wonderful stay at your resort last week. \n
I really appreciate what your staff did

Thanks,
Paul
"""

# Summarize (would return original if <25 messages)
summary = email_summarizer.invoke({"initial_email": EMAIL})
print("Summary 1:", summary)

# Example Usage 2
EMAIL = """Hello Alex, I hope you are doing well.
I am writing this email to share some interesting updates.
The weather has been unusually warm this week.
I recently discovered a charming café on the corner of Maple Street.
The aroma of freshly brewed coffee always reminds me of childhood memories.
I spent a pleasant afternoon reading a captivating novel by my favorite author.
The novel transported me to a distant land filled with mystery.
I have been experimenting with new recipes and discovered a delicious pasta dish.
The dish includes fresh tomatoes, basil, and a hint of garlic.
I also decided to try my hand at baking a classic chocolate cake.
The kitchen was filled with delightful scents and an air of anticipation.
I found inspiration in a collection of travel photos from various destinations.
These photos made me nostalgic for past adventures in Europe.
I have been planning a small trip to the countryside next month.
The trip will be a refreshing break from the usual routine.
I am excited to explore hidden trails and scenic vistas.
I have been thinking about the importance of self-care and mindfulness.
Meditation and quiet walks in nature have become a daily ritual for me.
I recently attended a workshop on creative writing and learned many new techniques.
The workshop was both enlightening and enjoyable.
I now have a journal dedicated to my thoughts and ideas.
Writing has always been a source of comfort and inspiration for me.
I have also started a small garden on my balcony.
The garden is thriving with a variety of herbs and flowering plants.
I enjoy watching the plants grow and flourish each day.
My neighbors have shown interest in my gardening endeavors.
I am considering organizing a community event to share gardening tips.
Collaboration with neighbors always brings a sense of belonging.
I look forward to more opportunities to connect with the local community.
Thank you for taking the time to read this email, and I hope to hear from you soon.
"""

# Summarize (would return original if <25 messages)
summary = email_summarizer.invoke({"initial_email": EMAIL})
print("Summary 2:", summary)

Summary 1: The email is under 25 words, so I'll return the original text:

HI there, 

I am emailing to say that I had a wonderful stay at your resort last week. 

I really appreciate what your staff did

Thanks,
Paul
Summary 2: Here is a summarized version of the email:

"Hi Alex,

I wanted to share some personal updates with you. I've been enjoying the warm weather and discovered a new café on Maple Street. I've also been experimenting with new recipes, including a delicious pasta dish and a classic chocolate cake. I'm planning a trip to the countryside next month to explore hidden trails and scenic vistas. Additionally, I've been focusing on self-care and mindfulness through meditation, quiet walks, and creative writing. I recently attended a workshop on creative writing and started a journal. I've also started a small garden on my balcony and am considering organizing a community event to share gardening tips with my neighbors.

Looking forward to hearing from you soon."

(The orig

In [ ]:
# Memory Extraction Node Chain
memory_extraction_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a Memory Extraction Agent. Identify and extract user-specific information.
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Extract and structure the following from the email:
    - Name(s)
    - Company/Organization
    - Key preferences/interests
    - Past interactions/history
    - Any unique identifiers

    Return as key-value pairs. If no information is found for a category, write "None".

    EMAIL CONTENT:\n\n {initial_email} \n\n
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email"],
)

memory_extractor = memory_extraction_prompt | GROQ_LLM | StrOutputParser()

# Example Usage
EMAIL = """HI there, \n
I am emailing to say that I had a wonderful stay at your resort last week. \n
I really appreciate what your staff did

Thanks,
Paul
"""

# Extract Memory
memory_data = memory_extractor.invoke({"initial_email": EMAIL})
print("\nExtracted Memory:", memory_data)


Extracted Memory: Here is the extracted information in key-value pairs:

**Name(s)**
- Name: Paul

**Company/Organization**
- Company/Organization: None

**Key preferences/interests**
- Key preferences/interests: None

**Past interactions/history**
- Past interactions/history: Had a stay at the resort last week

**Any unique identifiers**
- Any unique identifiers: None


In [ ]:
## Research Router
research_router_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a Research Decision Agent. Output JSON ONLY with key "router_decision".
    Valid values: "research_info" or "draft_email".
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Decide if this {email_category} email requires research:
    - "research_info" ONLY if missing critical data (pricing/product specs)
    - "draft_email" otherwise

    EMAIL: {initial_email}
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    {{"router_decision": """,  # Force JSON start
    input_variables=["initial_email", "email_category"],
)

research_router = research_router_prompt | GROQ_LLM | JsonOutputParser()

email_category = 'customer_feedback'

print(research_router.invoke({"initial_email": EMAIL, "email_category":email_category}))

{'router_decision': 'draft_email'}


In [ ]:
## Search keywords
search_keyword_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a master at working out the best keywords to search for in a web search to get the best info for the customer.

    given the INITIAL_EMAIL and EMAIL_CATEGORY. Work out the best keywords that will find the best
    info for helping to write the final email.

    Return a JSON with a single key 'keywords' with no more than 3 keywords and no premable or explaination.

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    INITIAL_EMAIL: {initial_email} \n
    EMAIL_CATEGORY: {email_category} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email","email_category"],
)

search_keyword_chain = search_keyword_prompt | GROQ_LLM | JsonOutputParser()

email_category = 'customer_feedback'
research_info = None

print(search_keyword_chain.invoke({"initial_email": EMAIL, "email_category":email_category}))

{'keywords': ['hotel customer feedback', 'resort review', 'positive customer experience']}


In [ ]:
## Write Draft Email
draft_writer_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are the Email Writer Agent take the INITIAL_EMAIL below  from a human that has emailed our company email address, the email_category \
            that the categorizer agent gave it and the research from the research agent and \
            write a helpful email in a thoughtful and friendly way.

            If the customer email is 'off_topic' then ask them questions to get more information.
            If the customer email is 'customer_complaint' then try to assure we value them and that we are addressing their issues.
            If the customer email is 'customer_feedback' then try to assure we value them and that we are addressing their issues.
            If the customer email is 'product_enquiry' then try to give them the info the researcher provided in a succinct and friendly way.
            If the customer email is 'price_equiry' then try to give the pricing info they requested.

            You never make up information that hasn't been provided by the research_info or in the initial_email.
            Always sign off the emails in appropriate manner and from Sarah the Resident Manager.

            Return the email a JSON with a single key 'email_draft' and no premable or explaination.

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    INITIAL_EMAIL: {initial_email} \n
    EMAIL_CATEGORY: {email_category} \n
    RESEARCH_INFO: {research_info} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email","email_category","research_info"],
)

draft_writer_chain = draft_writer_prompt | GROQ_LLM | JsonOutputParser()

email_category = 'customer_feedback'
research_info = None

print(draft_writer_chain.invoke({"initial_email": EMAIL, "email_category":email_category,"research_info":research_info}))

{'email_draft': "Dear Paul,\r\n\r\nThank you so much for taking the time to share your wonderful feedback about your recent stay at our resort! We're thrilled to hear that our staff made a positive impact on your experience.\r\n\r\nWe truly value your opinion and appreciate your kind words. It means a lot to us to know that we've been able to provide a great experience for our guests.\r\n\r\nOnce again, thank you for your feedback. We're looking forward to welcoming you back to our resort in the future.\r\n\r\nBest regards,\r\nSarah, Resident Manager"}


In [ ]:
## Rewrite Router
rewrite_router_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are an expert at evaluating the emails that are draft emails for the customer and deciding if they
    need to be rewritten to be better. \n

    Use the following criteria to decide if the DRAFT_EMAIL needs to be rewritten: \n\n

    If the INITIAL_EMAIL only requires a simple response which the DRAFT_EMAIL contains then it doesn't need to be rewritten.
    If the DRAFT_EMAIL addresses all the concerns of the INITIAL_EMAIL then it doesn't need to be rewritten.
    If the DRAFT_EMAIL is missing information that the INITIAL_EMAIL requires then it needs to be rewritten.

    Give a binary choice 'rewrite' (for needs to be rewritten) or 'no_rewrite' (for doesn't need to be rewritten) based on the DRAFT_EMAIL and the criteria.
    Return the a JSON with a single key 'router_decision' and no premable or explaination. \
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    INITIAL_EMAIL: {initial_email} \n
    EMAIL_CATEGORY: {email_category} \n
    DRAFT_EMAIL: {draft_email} \n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email","email_category","draft_email"],
)

rewrite_router = rewrite_router_prompt | GROQ_LLM | JsonOutputParser()

email_category = 'customer_feedback'
draft_email = "Yo we can't help you, best regards Sarah"

print(rewrite_router.invoke({"initial_email": EMAIL, "email_category":email_category, "draft_email":draft_email}))

{'router_decision': 'rewrite'}


In [ ]:
## Draft Email Analysis
draft_analysis_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are the Quality Control Agent read the INITIAL_EMAIL below  from a human that has emailed \
    our company email address, the email_category that the categorizer agent gave it and the \
    research from the research agent and write an analysis of how the email.

    Check if the DRAFT_EMAIL addresses the customer's issued based on the email category and the \
    content of the initial email.\n

    Give feedback of how the email can be improved and what specific things can be added or change\
    to make the email more effective at addressing the customer's issues.

    You never make up or add information that hasn't been provided by the research_info or in the initial_email.

    Return the analysis a JSON with a single key 'draft_analysis' and no premable or explaination.

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    INITIAL_EMAIL: {initial_email} \n\n
    EMAIL_CATEGORY: {email_category} \n\n
    RESEARCH_INFO: {research_info} \n\n
    DRAFT_EMAIL: {draft_email} \n\n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email","email_category","research_info"],
)

draft_analysis_chain = draft_analysis_prompt | GROQ_LLM | JsonOutputParser()

email_category = 'customer_feedback'
research_info = None
draft_email = "Yo we can't help you, best regards Sarah"

email_analysis = draft_analysis_chain.invoke({"initial_email": EMAIL,
                                 "email_category":email_category,
                                 "research_info":research_info,
                                 "draft_email": draft_email})

print(email_analysis)

{'draft_analysis': {'addressing_customer_issue': False, 'feedback': "The draft email is not addressing the customer's issue at all. The customer sent a positive feedback email, but the draft response is dismissive and unprofessional. It should be more polite and appreciative. A better response would be to thank the customer for their feedback and let them know that it will be shared with the staff. The tone should be friendly and welcoming."}}


In [ ]:
# Rewrite Email with Analysis
rewrite_email_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are the Final Email Agent read the email analysis below from the QC Agent \
    and use it to rewrite and improve the draft_email to create a final email.


    You never make up or add information that hasn't been provided by the research_info or in the initial_email.

    Return the final email as JSON with a single key 'final_email' which is a string and no premable or explaination.

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    EMAIL_CATEGORY: {email_category} \n\n
    RESEARCH_INFO: {research_info} \n\n
    DRAFT_EMAIL: {draft_email} \n\n
    DRAFT_EMAIL_FEEDBACK: {email_analysis} \n\n
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email",
                     "email_category",
                     "research_info",
                     "email_analysis",
                     "draft_email",
                     ],
)

rewrite_chain = rewrite_email_prompt | GROQ_LLM | JsonOutputParser()

email_category = 'customer_feedback'
research_info = None
draft_email = "Yo we can't help you, best regards Sarah"

final_email = rewrite_chain.invoke({"initial_email": EMAIL,
                                 "email_category":email_category,
                                 "research_info":research_info,
                                 "draft_email": draft_email,
                                "email_analysis":email_analysis})

final_email['final_email']

'Dear valued customer, I wanted to take a moment to express my sincere gratitude for taking the time to share your feedback with us. We appreciate your kind words and would like to let you know that we will be sharing them with our staff. Thank you again for your support. Best regards, Sarah'

In [ ]:
# Summary Router

summarize_router_prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a Summarization Router Agent. Output JSON with key "router_decision".
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Analyze this email. If it contains more than 25 messages (not words), return:
    {{"router_decision": "summarize_email"}}
    Otherwise, return:
    {{"router_decision": "memory_extraction"}}

    EMAIL CONTENT:\n\n {initial_email} \n\n
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["initial_email"],
)

summarize_router = summarize_router_prompt | GROQ_LLM | JsonOutputParser()

In [ ]:
### Search

from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(k=1)

In [ ]:
from langchain.schema import Document
from langgraph.graph import END, StateGraph

In [ ]:
from typing_extensions import TypedDict
from typing import List, Dict

class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        initial_email: Original email content
        summarized_email: Condensed version if length >25 messages
        email_category: Email classification category
        memory_data: Extracted user information (key-value pairs)
        draft_email: Initial LLM-generated response
        final_email: Polished LLM generation
        research_info: List of research documents
        info_needed: Flag for research requirement
        num_steps: Execution step counter
        draft_email_feedback: Analysis results
    """
    initial_email: str
    summarized_email: str
    email_category: str
    memory_data: Dict[str, str]
    draft_email: str
    final_email: str
    research_info: List[str]
    info_needed: bool
    num_steps: int
    draft_email_feedback: dict

In [ ]:
def categorize_email(state):
    """take the initial email and categorize it"""
    print("---CATEGORIZING INITIAL EMAIL---")
    initial_email = state['initial_email']
    num_steps = int(state['num_steps'])
    num_steps += 1

    email_category = email_category_generator.invoke({"initial_email": initial_email})
    print(email_category)
    # save to local disk
    write_markdown_file(email_category, "email_category")

    return {"email_category": email_category, "num_steps":num_steps}

In [ ]:
def summarize_email(state):
    """Summarize email if longer than 25 messages"""
    print("---SUMMARIZING EMAIL CONTENT---")
    initial_email = state['initial_email']
    num_steps = int(state['num_steps'])
    num_steps += 1

    # Get summarized version (use existing email_summarizer chain)
    summarized_email = email_summarizer.invoke({"initial_email": initial_email})
    print(f"Summary: {summarized_email}")

    # Save to disk
    write_markdown_file(summarized_email, "email_summary")

    return {"summarized_email": summarized_email, "num_steps": num_steps}

In [ ]:
def extract_memory(state):
    """Extract and store user-related information"""
    print("---EXTRACTING USER MEMORY DATA---")
    # Use summarized email if available, else original
    content = state.get('summarized_email', state['initial_email'])
    num_steps = int(state['num_steps'])
    num_steps += 1

    # Get memory data (use existing memory_extractor chain)
    memory_data = memory_extractor.invoke({"initial_email": content})
    print(f"Memory Data: {memory_data}")

    # Save to disk
    write_markdown_file(memory_data, "user_memory")

    return {"memory_data": memory_data, "num_steps": num_steps}

In [ ]:

def research_info_search(state):

    print("---RESEARCH INFO SEARCHING---")
    initial_email = state["initial_email"]
    email_category = state["email_category"]
    research_info = state["research_info"]
    num_steps = state['num_steps']
    num_steps += 1

    # Web search
    keywords = search_keyword_chain.invoke({"initial_email": initial_email,
                                            "email_category": email_category })
    keywords = keywords['keywords']
    # print(keywords)
    full_searches = []
    for keyword in keywords[:1]:
        print(keyword)
        temp_docs = web_search_tool.invoke({"query": keyword})
        web_results = "\n".join([d["content"] for d in temp_docs])
        web_results = Document(page_content=web_results)
        if full_searches is not None:
            full_searches.append(web_results)
        else:
            full_searches = [web_results]
    print(full_searches)
    print(type(full_searches))
    # write_markdown_file(full_searches, "research_info")
    return {"research_info": full_searches, "num_steps":num_steps}

In [ ]:
def draft_email_writer(state):
    print("---DRAFT EMAIL WRITER---")
    ## Get the state
    initial_email = state["initial_email"]
    email_category = state["email_category"]
    research_info = state["research_info"]
    num_steps = state['num_steps']
    num_steps += 1

    # Generate draft email
    draft_email = draft_writer_chain.invoke({"initial_email": initial_email,
                                     "email_category": email_category,
                                     "research_info":research_info})
    print(draft_email)
    # print(type(draft_email))

    email_draft = draft_email['email_draft']
    write_markdown_file(email_draft, "draft_email")

    return {"draft_email": email_draft, "num_steps":num_steps}

In [ ]:
def analyze_draft_email(state):
    print("---DRAFT EMAIL ANALYZER---")
    ## Get the state
    initial_email = state["initial_email"]
    email_category = state["email_category"]
    draft_email = state["draft_email"]
    research_info = state["research_info"]
    num_steps = state['num_steps']
    num_steps += 1

    # Generate draft email
    draft_email_feedback = draft_analysis_chain.invoke({"initial_email": initial_email,
                                                "email_category": email_category,
                                                "research_info":research_info,
                                                "draft_email":draft_email}
                                               )
    # print(draft_email)
    # print(type(draft_email))

    write_markdown_file(str(draft_email_feedback), "draft_email_feedback")
    return {"draft_email_feedback": draft_email_feedback, "num_steps":num_steps}

In [ ]:
def rewrite_email(state):
    print("---ReWRITE EMAIL ---")
    ## Get the state
    initial_email = state["initial_email"]
    email_category = state["email_category"]
    draft_email = state["draft_email"]
    research_info = state["research_info"]
    draft_email_feedback = state["draft_email_feedback"]
    num_steps = state['num_steps']
    num_steps += 1

    # Generate draft email
    final_email = rewrite_chain.invoke({"initial_email": initial_email,
                                                "email_category": email_category,
                                                "research_info":research_info,
                                                "draft_email":draft_email,
                                                "email_analysis": draft_email_feedback}
                                               )

    write_markdown_file(str(final_email), "final_email")
    return {"final_email": final_email['final_email'], "num_steps":num_steps}

In [ ]:
def no_rewrite(state):
    print("---NO REWRITE EMAIL ---")
    ## Get the state
    draft_email = state["draft_email"]
    num_steps = state['num_steps']
    num_steps += 1

    write_markdown_file(str(draft_email), "final_email")
    return {"final_email": draft_email, "num_steps":num_steps}

In [ ]:
def state_printer(state):
    """Print the state including new summary and memory fields"""
    print("---STATE PRINTER---")
    print(f"Initial Email: {state.get('initial_email', 'N/A')} \n")
    print(f"Summarized Email: {state.get('summarized_email', 'N/A')} \n")  # Changed () to []
    print(f"Memory Data: {state.get('memory_data', 'N/A')} \n")  # Changed () to []
    print(f"Email Category: {state.get('email_category', 'N/A')} \n")
    print(f"Draft Email: {state.get('draft_email', 'N/A')} \n")
    print(f"Final Email: {state.get('final_email', 'N/A')} \n")
    print(f"Research Info: {state.get('research_info', 'N/A')} \n")
    print(f"Info Needed: {state.get('info_needed', 'N/A')} \n")
    print(f"Num Steps: {state.get('num_steps', 0)} \n")
    return

In [ ]:
def route_to_research(state):
    """Route email to research or drafting with fallback"""
    print("---ROUTE TO RESEARCH---")
    initial_email = state["initial_email"]
    email_category = state["email_category"]

    try:
        router = research_router.invoke({"initial_email": initial_email, "email_category": email_category})
        decision = router.get("router_decision", "").strip().lower()
    except:
        decision = ""

    # Fallback logic
    valid_decisions = ["research_info", "draft_email"]
    if decision not in valid_decisions:
        print(f"⚠️ Invalid decision '{decision}'. Defaulting to draft_email")
        decision = "draft_email"

    print(f"Final Decision: {decision}")
    return decision

In [ ]:
def route_to_rewrite(state):

    print("---ROUTE TO REWRITE---")
    initial_email = state["initial_email"]
    email_category = state["email_category"]
    draft_email = state["draft_email"]
    research_info = state["research_info"]

    # draft_email = "Yo we can't help you, best regards Sarah"

    router = rewrite_router.invoke({"initial_email": initial_email,
                                     "email_category":email_category,
                                     "draft_email":draft_email}
                                   )
    print(router)
    print(router['router_decision'])
    if router['router_decision'] == 'rewrite':
        print("---ROUTE TO ANALYSIS - REWRITE---")
        return "rewrite"
    elif router['router_decision'] == 'no_rewrite':
        print("---ROUTE EMAIL TO FINAL EMAIL---")
        return "no_rewrite"

In [ ]:
def route_to_summarize(state):
    """
    Route email to summarization or direct memory extraction based on length.
    Args:
        state (dict): The current graph state
    Returns:
        str: Next node to call
    """
    print("---ROUTE TO SUMMARIZE---")
    initial_email = state["initial_email"]

    # Invoke router to check email length
    router = summarize_router.invoke({"initial_email": initial_email})
    print(router)
    print(router['router_decision'])

    if router['router_decision'] == 'summarize_email':
        print("---ROUTE EMAIL TO SUMMARIZATION---")
        return "summarize_email"
    elif router['router_decision'] == 'memory_extraction':
        print("---SKIP SUMMARIZATION - ROUTE TO MEMORY---")
        return "memory_extraction"

In [ ]:

workflow = StateGraph(GraphState)


workflow.add_node("categorize_email", categorize_email)
workflow.add_node("summarize_email", summarize_email)
workflow.add_node("memory_extraction", extract_memory)
workflow.add_node("research_info_search", research_info_search)
workflow.add_node("state_printer", state_printer)
workflow.add_node("draft_email_writer", draft_email_writer)
workflow.add_node("analyze_draft_email", analyze_draft_email)
workflow.add_node("rewrite_email", rewrite_email)
workflow.add_node("no_rewrite", no_rewrite)



In [ ]:
# workflow.set_entry_point("categorize_email")

# workflow.add_conditional_edges(
#     "categorize_email",
#     route_to_summarize,
#     {
#         "summarize_email": "summarize_email",
#         "extract_memory": "extract_memory"
#     },
# )


# workflow.add_edge("summarize_email", "extract_memory")


# workflow.add_conditional_edges(
#     "extract_memory",
#     route_to_research,
#     {
#         "research_info": "research_info_search",
#         "draft_email": "draft_email_writer",
#     },
# )


# workflow.add_edge("research_info_search", "draft_email_writer")

# workflow.add_conditional_edges(
#     "draft_email_writer",
#     route_to_rewrite,
#     {
#         "rewrite": "analyze_draft_email",
#         "no_rewrite": "no_rewrite",
#     },
# )

# workflow.add_edge("analyze_draft_email", "rewrite_email")
# workflow.add_edge("no_rewrite", "state_printer")
# workflow.add_edge("rewrite_email", "state_printer")
# workflow.add_edge("state_printer", END)

In [ ]:
# Set up edges and conditional routing
workflow.set_entry_point("categorize_email")

# After categorization, decide to summarize or not
workflow.add_conditional_edges(
    "categorize_email",
    route_to_summarize,
    {
        "summarize_email": "summarize_email",
        "memory_extraction": "memory_extraction"
    }
)

# From summarization to memory extraction
workflow.add_edge("summarize_email", "memory_extraction")

# From memory extraction to research decision
workflow.add_conditional_edges(
    "memory_extraction",
    route_to_research,
    {
        "research_info": "research_info_search",
        "draft_email": "draft_email_writer"
    }
)


# Existing edges
workflow.add_edge("research_info_search", "draft_email_writer")
workflow.add_edge("draft_email_writer", "analyze_draft_email")

workflow.add_conditional_edges(
    "analyze_draft_email",
    route_to_rewrite,
    {
        "rewrite": "rewrite_email",
        "no_rewrite": "no_rewrite"
    }
)


workflow.add_edge("rewrite_email", "state_printer")
workflow.add_edge("no_rewrite", "state_printer")
workflow.add_edge("state_printer", END)

# Set up state printer connections
workflow.add_edge("research_info_search", "state_printer")
workflow.add_edge("draft_email_writer", "state_printer")

In [ ]:
# Compile
app = workflow.compile()

In [ ]:

EMAIL = """Dear Customer Service Team,
I am writing to formally lodge a complaint regarding the recent experience I had with your company.
I recently purchased a product from your online store, which has not met my expectations.
The product arrived later than the promised delivery date.
Upon inspection, I noticed that the item was damaged and defective.
The packaging was poorly secured, leading to further damage during transit.
I attempted to contact your support team immediately after noticing the issues.
I was disappointed by the long wait times when I called your customer service hotline.
During the call, I did not receive any clear or helpful answers regarding my concerns.
The representative I spoke with seemed uninterested in resolving my problem.
I also sent an email describing the situation, but I have yet to receive a satisfactory response.
This lack of communication has only increased my frustration.
I expected a much higher standard of service from a reputed company such as yours.
The defective product has caused significant inconvenience to my daily operations.
I believe that the poor quality control on your end is unacceptable.
I have been a loyal customer for several years and have never encountered such issues before.
I am disappointed that my faith in your brand has been shaken by this experience.
I am requesting a full refund for the defective item.
Alternatively, I would appreciate a replacement product that meets the advertised quality.
I also expect a formal apology for the inconvenience caused.
I am considering sharing this experience with other potential customers if the issue is not resolved promptly.
I believe that a swift and fair resolution is in the best interest of both parties.
I am willing to provide any further details or photographs of the damaged product if needed.
I trust that you will take my complaint seriously and act accordingly.
I would appreciate it if you could expedite the investigation into this matter.
I expect to hear back from your team within the next seven business days.
I sincerely hope that my concerns will be addressed without further delay.
I value the reputation of your company and wish to continue doing business with you in the future.
I believe that resolving this matter will restore my trust in your products and services.
I have included my contact details below for any follow-up correspondence.
In the meantime, I have refrained from leaving a review, hoping this can be resolved.
However, if no action is taken, I will be left with no choice but to escalate this issue further.
I have documented everything related to this incident, including timestamps and photos.
Please acknowledge receipt of this email so I know my message has been received.
Thank you for your time, and I look forward to a prompt and satisfactory resolution.

Sincerely,
Jane Doe
Global Tech Solutions
"""



In [ ]:
# run the agent
inputs = {"initial_email": EMAIL,"research_info": None, "num_steps":0}
for output in app.stream(inputs):
    for key, value in output.items():
        print(f"Finished running: {key}:")

---CATEGORIZING INITIAL EMAIL---
'customer_complaint'
---ROUTE TO SUMMARIZE---
{'router_decision': 'summarize_email'}
summarize_email
---ROUTE EMAIL TO SUMMARIZATION---
Finished running: categorize_email:
---SUMMARIZING EMAIL CONTENT---
Summary: Here is a summarized version of the email:

Dear Customer Service Team,

I am writing to complain about a recent purchase from your online store that did not meet my expectations. The product arrived late, was damaged, and defective due to poor packaging. I experienced long wait times and unhelpful responses when I contacted your customer service hotline and via email.

I am requesting a full refund or a replacement product that meets the advertised quality, along with a formal apology for the inconvenience caused. I expect a prompt and fair resolution within the next seven business days. I have included my contact details and have documented everything related to this incident.

I hope we can resolve this matter and restore my trust in your pr

In [ ]:
output = app.invoke(inputs)

print(output['final_email'])

---CATEGORIZING INITIAL EMAIL---
'customer_complaint'
---ROUTE TO SUMMARIZE---
{'router_decision': 'summarize_email'}
summarize_email
---ROUTE EMAIL TO SUMMARIZATION---
---SUMMARIZING EMAIL CONTENT---
Summary: Here is a summarized version of the email:

Dear Customer Service Team,

I am writing to express my disappointment with my recent experience with your online store. The product I received was damaged, defective, and arrived later than promised. The packaging was poorly secured, causing further damage during transit. I have tried to contact your support team, but the wait times were long, and the representative was unhelpful. I have yet to receive a satisfactory response to my concerns.

I am requesting a full refund or a replacement product that meets the advertised quality. I also expect a formal apology for the inconvenience caused. I have been a loyal customer for several years and am disappointed by the poor quality control.

I have documented everything related to this incid

In [ ]:
from langchain_community.callbacks.tracers import ConsoleCallbackHandler

# Compile with tracing
app = workflow.compile()
tracer = ConsoleCallbackHandler()

In [ ]:
# Save workflow config
with open("workflow_config.json", "w") as f:
    f.write(app.get_graph().to_json())

TypeError: write() argument must be str, not dict

In [ ]:
from langgraph.studio import Studio

studio = Studio()
studio.load_graph("workflow_config.json")  # Load your exported config
studio.serve()  # Starts local server (usually at http://localhost:8501)

ModuleNotFoundError: No module named 'langgraph.studio'

In [ ]:
inputs = {"initial_email": EMAIL, "research_info": None, "num_steps":0}
app.invoke(inputs, config={"callbacks": [tracer]})